# Fast groupby operations

- hide: true
- category: [python, pandas]

In [1]:
from imports import *

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
pd.read_csv

'/Users/fgu/dev/projects/blog/_notebooks'

In [79]:
path = '/Users/fgu/tmp/data_777.parquet'
df = pd.read_parquet(path)
print(df.shape)
df.head()

(115197, 21)


,user_id,transaction_date,amount,transaction_description,merchant_name,tag,gender,transaction_id,bank,account_created,account_type,postcode,merchant_business_line,ym,account_id,year_of_birth,user_registration_date,salary_range,credit_debit,account_last_refreshed,latest_balance
6553,14777,2012-10-10,5.00,new southern railw cd 7715,southern rail,public transport,m,2994290,lloyds bank,2013-01-07,current,sw11 5,southern rail,201210,247061,1986.0,2013-01-07,NaN,debit,2015-01-13 05:19:00,729.59
6554,14777,2012-10-10,11.10,new southern railw cd 7715,southern rail,public transport,m,2994291,lloyds bank,2013-01-07,current,sw11 5,southern rail,201210,247061,1986.0,2013-01-07,NaN,debit,2015-01-13 05:19:00,729.59
6555,14777,2012-10-10,26.91,ticketscript*14064 cd 7715,no merchant,public transport,m,2994292,lloyds bank,2013-01-07,current,sw11 5,unknown merchant,201210,247061,1986.0,2013-01-07,NaN,debit,2015-01-13 05:19:00,729.59
6556,14777,2012-10-10,83.20,selfserve ticket cd 2412,south west trains,public transport,m,2994293,lloyds bank,2013-01-07,current,sw11 5,south west trains,201210,247061,1986.0,2013-01-07,NaN,debit,2015-01-13 05:19:00,729.59
6557,14777,2012-10-11,19.00,liv*livingsocial cd 7715,no merchant,dining and drinking,m,2994289,lloyds bank,2013-01-07,current,sw11 5,unknown merchant,201210,247061,1986.0,2013-01-07,NaN,debit,2015-01-13 05:19:00,729.59


## Applications

Baseline function to filter out users with minumum number of transactions and spend in each month.

In [41]:
def min_txns_and_spend(df, min_txns=10, min_spend=300):
    """At least 5 monthly debits totalling GBP200.
    Drops first and last month for each user due to possible partial data.
    """
    cols = ['user_id', 'transaction_date', 'amount']
    spend = df.loc[df.amount > 0, cols]
    spend = spend.set_index('transaction_date')
    spend['ym'] = df.transaction_date.dt.to_period('M')

    g = spend.groupby('user_id')
    spend['first_month'] = g.ym.transform(min)
    spend['last_month'] = g.ym.transform(max)
    spend = spend[(spend.ym != spend.first_month)
                  & (spend.ym != spend.last_month)]

    mins = (spend.groupby('user_id')
            .resample('M').amount.agg(['count', 'sum'])
            .groupby('user_id').min())
    mask = (mins >= (min_txns, min_spend)).all(1)
    users = mins[mask].index
    return df[df.user_id.isin(users)]

%timeit min_txns_and_spend(df)

526 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [164]:
%%timeit

"""
At least 5 monthly debits totalling GBP200.
Drops first and last month for each user due to possible partial data.
"""
data = df[['user_id', 'ym', 'amount']]

# keep debits
data = data[data.amount > 0]

# remove first and last month from each user
g = data.groupby('user_id')
first_month = g.ym.transform(min)
last_month = g.ym.transform(max)
data[(data.ym > first_month) & (data.ym < last_month)]

# calculate more stuff
# g = data.groupby(['user_id', 'ym'])
# large_spend = g.amount.sum() > 200
# large_spend.groupby('user_id').sum()
# large_spend.groupby('user_id').iloc[1:-1]

10.9 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Basics

### Boolean comparisons

In [66]:
df = sns.load_dataset('iris')

In [67]:
%timeit df.sepal_width > 3

105 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [68]:
%timeit df.sepal_width.values > 0

6.11 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Given the above, the below is rather surprising:

In [69]:
%timeit df[df.sepal_width > 3]

335 µs ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [72]:
%timeit df[df.sepal_width.values > 3]

148 µs ± 6.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Group means

From https://cmdlinetips.com/2019/05/how-to-implement-pandas-groupby-operation-with-numpy/



In [17]:
df = sns.load_dataset('iris')

In [18]:
%timeit df.groupby('species').sepal_length.mean()

481 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%%timeit

spec = df.species.values
sl = df.sepal_length.values
groups = df.species.unique()
[(group, np.mean(sl[spec == group])) for group in groups]

111 µs ± 7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
